In [ ]:
import os
import json

import pandas as pd
import geopandas as gpd

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import rc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
cwd = os.getcwd()
REPORT_PATH = os.path.join(cwd, "..", "..", "reports", "03_D2.4_appendix1")
DATA_PATH = os.path.join(cwd, "..", "..", "data", "input", "processed")
VAR_DF_PATH = os.path.join(cwd, "..", "..", "data", "input", "raw", "vars_list_details_and_tags.xlsx")
SHP_PATH = os.path.join(cwd, "..", "..", "data", "input", "processed", "shapefiles")

SAVE_PATH = os.path.join(cwd, "..", "..", "data", "output", "api_data")

In [ ]:
regions_df = gpd.read_file(os.path.join(SHP_PATH, "LAU.shp"))

In [ ]:
regions_df.drop(columns= ["year"], inplace=True)

In [ ]:
def processes_data(data_df):

    # merge dfs
    final_df = pd.merge(
        regions_df,
        data_df,
        left_on="prnt_code",
        right_on="reg_code",
        how="right",
    )

    final_df.drop(
        columns=[
            "reg_code",
            "name"
        ],
        inplace=True,
    )

    return final_df

In [ ]:
clm_vars = {"annual mean mean temperature": "degree celsius",
            "annual mean precipitation": "mm/day",
            "annual total precipitation": "mm",
            "annual mean maximum temperature": "degree celsius",
            "annual mean minimum temperature": "degree celsius",
            "annual mean temperature cooling degree days": "sum degree celsius",
            "annual maximum temperature cooling degree days": "sum degree celsius",
            "annual minimum temperature cooling degree days": "sum degree celsius",
            "annual mean temperature heating degree days": "sum degree celsius",
            "annual maximum temperature heating degree days": "sum degree celsius",
            "annual minimum temperature heating degree days": "sum degree celsius"}

In [ ]:
#NOTE: data shown only every 10 years
years = ['2020', '2030', '2040', '2050', '2060', '2070', '2080', '2090', '2099']

In [ ]:
nl = '\n'
latex_nl = '\\\\'
esc_nl = '\\'

In [ ]:
replacements = {'begin{tabular}': 'begin{tabularx}{\\textwidth}',
                'end{tabular}': 'end{tabularx}',
                 "& 0 \\\\": "",
                "_": "\_",
                '\\\\': '\\\\\midrule',
                "{\\textwidth}{| X | X |}\n": "{\\textwidth}{| X | X |}\n\midrule"

               }

def get_latex_table(df):
    # prepare column_format
    col_format = "| X |"
    for i in range(len(df.columns)):
        col_format = f"{col_format} X |"
    
    # pd to latex
    table = df.style.to_latex(column_format=col_format, 
                                             position_float="centering",
                                             hrules=False,
                                             environment = "table*", 
                                             position="h"
                                             )
    # replace tabular with tabularx and add width=textwidth
    for key, value in replacements.items():
        table = table.replace(key, value)
        
    return table 

In [ ]:
# tables
for var_name, var_unit in clm_vars.items():
    table_dict = {"Unit of measure": var_unit,
                 "Data years": "Yearly data from 2020 to 2099",
                 "Original resolution": "NUTS3",
                  "Further disaggregated?": "yes",
                 "Disaggregation method": "Same value as parent NUTS3 region to its LAU regions"}
    
    table = pd.DataFrame.from_dict(table_dict, orient="index")
    
    with open(os.path.join(REPORT_PATH,  "tables", f"{var_name[0].upper()}{var_name[1:]}.tex"), "w") as f:

        latex_table = get_latex_table(table)
        f.write(latex_table)
        f.write(nl)

In [ ]:
# BOX plots
for var_name, var_unit in clm_vars.items():
    
    data_df_list = []
    for year in years:
        data_df_year = pd.read_csv(os.path.join(DATA_PATH, var_name, year, "data.csv"))
        data_df_year["year"] = int(year)
        data_df_list.append(data_df_year)
        
    data_df = pd.concat(data_df_list)
    data_df = processes_data(data_df)

    for country_code in ["DE", "PL", "ES"]:
        sub_df = data_df[data_df["prnt_code"].str.startswith(country_code)]

        fig = plt.figure(figsize=(7, 5))
        gs = fig.add_gridspec(3, 1, wspace=0, hspace=0.3)

        ax1 = plt.subplot(gs[:1, :]) #-------------------------------
        
        rcp_sub_df = sub_df[sub_df["climate_experiment"] == "RCP2.6"]
        
        sns.boxplot(x = rcp_sub_df["year"] ,
                    y = rcp_sub_df["value"],
                    width=0.3, ax=ax1)
        
        ax1.set_title("Climate scenario: RCP2.6")
        
        ax1.set_xticklabels([])
        ax1.set_xlabel("")
        ax1.set_ylabel("")
        
        ax2 = plt.subplot(gs[1:2, :]) #-------------------------------
        
        rcp_sub_df = sub_df[sub_df["climate_experiment"] == "RCP4.5"]
        
        sns.boxplot(x = rcp_sub_df["year"] ,
                    y = rcp_sub_df["value"],
                    width=0.3, ax=ax2)
        
        ax2.set_xticklabels([])
        ax2.set_xlabel("")
        ax2.set_ylabel(f"[{var_unit}]", fontsize=13)
        
        ax2.set_title("Climate scenario: RCP4.5")
        
        ax3 = plt.subplot(gs[2:, :]) #-------------------------------
        
        rcp_sub_df = sub_df[sub_df["climate_experiment"] == "RCP8.5"]
        
        sns.boxplot(x = rcp_sub_df["year"] ,
                    y = rcp_sub_df["value"],
                    width=0.3, ax=ax3)
        
        ax3.set_xlabel("year", fontsize=13)
        ax3.set_ylabel("")
        
        ax3.set_title("Climate scenario: RCP8.5")
        
        plt.savefig(os.path.join(REPORT_PATH, 
                                 "figures", 
                                 f"{var_name[0].upper()}{var_name[1:]}_{country_code}_box.png"),
                   format='png', bbox_inches="tight", dpi=200)
    

In [ ]:
# maps
for var_name, var_unit in clm_vars.items():
    
    data_df_2020 = pd.read_csv(os.path.join(DATA_PATH, var_name, "2020", "data.csv"))
    data_df_2099 = pd.read_csv(os.path.join(DATA_PATH, var_name, "2099", "data.csv"))
        
    data_df_2020 = processes_data(data_df_2020)
    data_df_2099 = processes_data(data_df_2099)
    
    for country_code in ["DE", "PL", "ES"]:
        
        country_df_2020 = data_df_2020[data_df_2020["prnt_code"].str.startswith(country_code)]
        country_df_2099 = data_df_2099[data_df_2099["prnt_code"].str.startswith(country_code)]

        fig = plt.figure(figsize=(5, 7))
        gs = fig.add_gridspec(3, 2, wspace=0, hspace=0)
        
        # set min and max values across all subplots
        val_list = list(country_df_2020['value']) + list(country_df_2099['value'])
        vmin, vmax = min(val_list), max(val_list) 
        
        ax1 = plt.subplot(gs[:1, :1]) # RCP2.6, year 2020-------------------------------
        
        _sub_df = country_df_2020[country_df_2020["climate_experiment"] == "RCP2.6"]
        
        _sub_df.plot(column='value', 
                     cmap='Blues', 
                     linewidth=0, 
                     ax=ax1, 
                     edgecolor='none', 
                     norm=plt.Normalize(vmin=vmin, vmax=vmax))
        
        ax1.set_ylabel("Climate scenario:\nRCP2.6", fontsize=7)
        ax1.tick_params(
            axis='x',          
            which='both',      
            bottom=False,      
            top=False,        
            labelbottom=False) 
        
        ax1.tick_params(
            axis='y',          
            which='both',      
            left=False,     
            right=False,         
            labelleft=False) 
        
        ax1.set_frame_on(False)

        ax2 = plt.subplot(gs[:1, 1:]) # RCP2.6, year 2099-------------------------------
        _sub_df = country_df_2099[country_df_2099["climate_experiment"] == "RCP2.6"]
        
        _sub_df.plot(column='value', 
                     cmap='Blues', 
                     linewidth=0, 
                     ax=ax2, 
                     edgecolor='none', 
                     norm=plt.Normalize(vmin=vmin, vmax=vmax))
        
        ax2.tick_params(
            axis='x',          
            which='both',      
            bottom=False,      
            top=False,        
            labelbottom=False) 
        
        ax2.tick_params(
            axis='y',          
            which='both',      
            left=False,     
            right=False,         
            labelleft=False) 
        
        ax2.set_frame_on(False)

        
        ax3 = plt.subplot(gs[1:2, :1]) # RCP4.5, year 2020-------------------------------
        _sub_df = country_df_2020[country_df_2020["climate_experiment"] == "RCP4.5"]
        
        _sub_df.plot(column='value', 
                     cmap='Blues', 
                     linewidth=0, 
                     ax=ax3, 
                     edgecolor='none', 
                     norm=plt.Normalize(vmin=vmin, vmax=vmax))
        
        ax3.set_ylabel("Climate scenario:\nRCP4.5", fontsize=7)
        ax3.tick_params(
            axis='x',          
            which='both',      
            bottom=False,      
            top=False,        
            labelbottom=False) 
        
        ax3.tick_params(
            axis='y',          
            which='both',      
            left=False,     
            right=False,         
            labelleft=False) 
        
        ax3.set_frame_on(False)
        
        ax4 = plt.subplot(gs[1:2, 1:]) # RCP4.5, year 2099-------------------------------
        _sub_df = country_df_2099[country_df_2099["climate_experiment"] == "RCP4.5"]
        
        _sub_df.plot(column='value', 
                     cmap='Blues', 
                     linewidth=0, 
                     ax=ax4, 
                     edgecolor='none', 
                     norm=plt.Normalize(vmin=vmin, vmax=vmax))
        
        ax4.tick_params(
            axis='x',          
            which='both',      
            bottom=False,      
            top=False,        
            labelbottom=False) 
        
        ax4.tick_params(
            axis='y',          
            which='both',      
            left=False,     
            right=False,         
            labelleft=False) 
        
        ax4.set_frame_on(False)
        
        sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
        sm._A = []
        axins = inset_axes(ax4,
                    width="4%",  
                    height="100%",
                    loc='center right',
                    borderpad=0
                   )
        clb=fig.colorbar(sm, 
                     cax=axins, 
                     orientation="vertical",
                    )
        clb.ax.tick_params(labelsize=6)
        clb.ax.set_title(f"[{var_unit}]", rotation = 90, x=2.5, y=1, fontsize=6)
        
        ax5 = plt.subplot(gs[2:3, :1]) # RCP8.5, year 2020-------------------------------
        _sub_df = country_df_2020[country_df_2020["climate_experiment"] == "RCP8.5"]
        
        _sub_df.plot(column='value', 
                     cmap='Blues', 
                     linewidth=0, 
                     ax=ax5, 
                     edgecolor='none', 
                     norm=plt.Normalize(vmin=vmin, vmax=vmax))
        
        ax5.set_ylabel("Climate scenario:\nRCP8.5", fontsize=7)
        ax5.set_xlabel("Year: 2020", fontsize=7)
        ax5.tick_params(
            axis='x',          
            which='both',      
            bottom=False,      
            top=False,        
            labelbottom=False) 
        
        ax5.tick_params(
            axis='y',          
            which='both',      
            left=False,     
            right=False,         
            labelleft=False) 
        
        ax5.set_frame_on(False)
        
        ax6 = plt.subplot(gs[2:3, 1:]) # RCP8.5, year 2099-------------------------------
        _sub_df = country_df_2099[country_df_2099["climate_experiment"] == "RCP8.5"]
        
        _sub_df.plot(column='value', 
                     cmap='Blues', 
                     linewidth=0, 
                     ax=ax6, 
                     edgecolor='none', 
                     norm=plt.Normalize(vmin=vmin, vmax=vmax))
        
        ax6.set_xlabel("Year: 2099", fontsize=7)
        ax6.tick_params(
            axis='x',          
            which='both',      
            bottom=False,      
            top=False,        
            labelbottom=False) 
        
        ax6.tick_params(
            axis='y',          
            which='both',      
            left=False,     
            right=False,         
            labelleft=False) 
        
        ax6.set_frame_on(False)
        
        plt.savefig(os.path.join(REPORT_PATH, 
                                 "figures", 
                                 f"{var_name[0].upper()}{var_name[1:]}_{country_code}_map.png"),
                   format='png', bbox_inches="tight", dpi=200)